In [24]:
import os
from pathlib import Path


os.chdir(Path.cwd().parent)

In [25]:
import numpy as np

from src.rayleigh_quotient import rayleigh_quotient
from src.rayleigh_quotient_gradient import rayleigh_quotient_gradient
from src.rayleigh_quotient_hessian import rayleigh_quotient_hessian, rayleigh_quotient_hessian_naive
from src.utils.linalg import (
    generate_normalized_vector, generate_supersymmetric_tensor, normalize_vector, projection
)
from src.utils.testing import timer


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
def test_rayleigh_quotient_hessian() -> None:
    k, n = 2, 2
    A = generate_supersymmetric_tensor(k, n)
    x = generate_normalized_vector(n)
    proj = projection(x)
    hess_true = 2 * proj @ (A - rayleigh_quotient(A, x) * np.eye(n)) @ proj
    hess_estimated = rayleigh_quotient_hessian(A, x)
    assert np.allclose(hess_true, hess_estimated, atol=1e-4)


In [27]:
test_rayleigh_quotient_hessian()

In [28]:
def G_t_derivative_1(x: np.ndarray, xi: np.ndarray, t: int = 0) -> np.ndarray:
    assert t == 0  # t != 0 later
    x_norm = np.linalg.norm(x)
    return xi / x_norm - ((x.T @ xi) * x) / x_norm**3

def G_t_derivative_2(x: np.ndarray, xi: np.ndarray, t: int = 0) -> np.ndarray:
    assert t == 0  # t != 0 later
    x_norm = np.linalg.norm(x)
    xi_norm = np.linalg.norm(xi)
    x_xi = x.T @ xi
    return (3 * x_xi**2 * x) / x_norm**5 - (xi_norm**2 * x + 2 * x_xi * xi) / x_norm**3

def test_hessian_approximation(n: int, k: int, eps: float = 1e-6) -> None:
    """
    Testing Riemannian hessian correctness with second order expansion of Rayleigh quotient.
    See https://www.nicolasboumal.net/book/IntroOptimManifolds_Boumal_2023.pdf, Chapter 5.9
        for implementation details.
    """

    print(f"test hessian approximation with {n=}, {k=}")
    print()

    errors = []
    for _ in range(10):
        A = generate_supersymmetric_tensor(n, k)
        for __ in range(10):
            x = generate_normalized_vector(n)
            rq = rayleigh_quotient(A, x)
            grad = rayleigh_quotient_gradient(A, x)
            hess = rayleigh_quotient_hessian(A, x)
            for ___ in range(10):
                t = generate_normalized_vector(n)
                deriv1 = G_t_derivative_1(x, t)
                deriv2 = G_t_derivative_2(x, t)
                rq1 = rayleigh_quotient(A, normalize_vector(x + eps * t))
                rq2 = rayleigh_quotient(A, normalize_vector(x - eps * t))
                hess_vec = deriv1.T @ hess @ deriv1
                grad_vec = grad.T @ deriv2
                lhs = (rq1 + rq2 - 2 * rq) / eps**2 - grad_vec
                rhs = hess_vec
                error = np.linalg.norm(lhs - rhs)
                errors.append(error)

    print(f"max error = {max(errors)}")
    print(f"mean error = {np.mean(errors)}")

In [30]:
test_hessian_approximation(n=6, k=6, eps=1e-4)

test hessian approximation with n=6, k=6

max error = 0.595321700166096
mean error = 0.1442675795696444
